# Common Pitfall in Elevation differencing

## Data from different sources

The lidar data we have used so far are acquired from the same source (QSI). This made the DEM differencing to be quiet straightforward because the crs, resolution and extent are the same. At times, the elevation data might be from diffeerence sources. It will thus require extra effort of resampling and reprojection to accurately derive snow depth. 
Let's do DEM differencing again but using snow-on and snow-off DEM from different sources. The snow-on data is an 0.5m resolution raster acquired by  QSI over GrandMesa in 2020. A 3m DEM data acquired by ASO in 2016 will be used as the snow-off DEM. 

As a first step, we have to downsample the QSI data to the resolution of the ASO.

In [ ]:
!gdal_translate  -tr 3, 3 ./input/QSI_0.5m_GrandMesa13Feb_Winter_DEM.tif ./Results/QSI_3m_GrandMesa13Feb_Winter_DEM.tif

In [ ]:
#check the raster metadata
!gdalinfo ./Results/QSI_3m_GrandMesa13Feb_Winter_DEM.tif

In [ ]:
#read the data as arrays
snow_on_dem = rioxarray.open_rasterio('./Results/QSI_3m_GrandMesa13Feb_Winter_DEM.tif', masked =True)
snow_on_dem.hvplot.image(x= 'x', y = 'y', rasterize = True, cmap = 'viridis')

In [ ]:
#read the snow-off DEMs
snow_free_dem = rioxarray.open_rasterio('./input/ASO_3M_GrandMesa_Summer_DEM.tif',
            masked=True)
snow_free_dem.hvplot.image(x= 'x', y = 'y', rasterize = True, cmap = 'viridis')

The extent and crs of the snow-off DEM is different from the snow-on. Let's match the snow-off DEM with the snow-on DEM

In [ ]:
snow_free_dem_match = snow_free_dem.rio.reproject_match(snow_on_dem)
snow_free_dem_match.hvplot.image(x= 'x', y = 'y', rasterize = True, cmap = 'viridis')

In [ ]:
snow_depth = snow_on_dem - snow_free_dem_match
snow_depth.hvplot.image(x= 'x', y = 'y', rasterize = True, cmap = 'viridis')

About 17 m snow depth.... What's the issue?

The snow-on and snow-free DEM are referenced to different datum, so we need to do vertical datum transformation. Matching the vertical datum of data from different sources is important for elevation data. 
The vertical datum of ASO is in ellipsoid (WGS 84)while QSI is in geoid (NAVD 88). Let's transform ASO to same datum as QSI. 

QSI is in UTM Zone 12 North. Horizontal Datum used is NAD83 (2011). NAD 83 is a 3-D reference system, it also serves as the horizontal control datum for the United States, Canada, Greenland and Central America using the Geodetic Reference System 1980 (GRS 80) ellipsoid, an earth-centered reference ellipsoid. [See this link for more reading](https://vdatum.noaa.gov/docs/datums.html). The vertical datum is NAVD88 (Geoid12b)

<figure>
<img src = 'https://i.imgur.com/CbvMuMB.jpg' style="width:100%">
<figcaption align =  'center'><b>Relationship between Clarke 1866, WGS 84, GRS 80 and Geoid (left).Relationship between elliposid, Geoid and Orthometric Height (Right). Source: NOAA</b></figcaption>
</figure>





In [ ]:
!gdalsrsinfo -o proj4 ./input/ASO_3M_GrandMesa_Summer_DEM.tif

In [ ]:
!gdalsrsinfo -o proj4 ./Results/QSI_3m_GrandMesa13Feb_Winter_DEM.tif

In [ ]:
#coordinate transformation
!gdalwarp -s_srs "+proj=utm +zone=13 +datum=WGS84 +units=m +no_defs" -t_srs "+proj=utm +zone=12 +ellps=GRS80 +units=m +no_defs +geoidgrids=egm96_15.gtx" ./input/ASO_3M_GrandMesa_Summer_DEM.tif ./Results/ASO_3M_GrandMesa_Summer_DEM_geoid.tif

In [ ]:
#read the snow-off DEMs
snow_free_dem = rioxarray.open_rasterio('./Results/ASO_3M_GrandMesa_Summer_DEM_geoid.tif',
            masked=True)
            
#reproject the ASO DEM to match the QSI DEM
snow_free_dem_match = snow_free_dem.rio.reproject_match(snow_on_dem)

In [ ]:
snow_depth = snow_on_dem - snow_free_dem_match
snow_depth = snow_depth.where((snow_depth > 0) & (snow_depth < 3))
snow_depth.hvplot.image(x= 'x', y = 'y', rasterize = True, cmap = 'viridis')

## Coregistion

Another potential source of error for lidar differencing is coregristration. If the differenced value still doesn't make sense after matching the crs and extent, it might be a misregistration issue.

## Available Data for Lidar Remote Sensing

Various lidar data has been acquired to monitory snow over the western US.
1. 2013 - 2019: ASO data
2. 2020 Quantunmn data over some selected areas.
3. 2021 Quantumn data over selected areas
4. 2022: Riegl flight over MCS

For the 2020 and 2021 data we have lidar data including 
The lidar data we have include:

1. DEMs (SnowOn and SnowFree)
2. DSMs (SnowOn and SnowFree)
3. Vegetation Height (SnowOn and SnowFree)
4. Snow Depth
5. Intensity Images (SnowOn and SnowFree)

![](https://i.imgur.com/Y5gsOFO.jpg)

References:
1. Earthdatascience